# AI Voice Assistant Pipeline with Low Latency

This notebook demonstrates the implementation of an AI voice assistant pipeline using Whisper for voice-to-text, the Mistral LLM for generating responses, and a Text-to-Speech (TTS) model with tunable parameters for voice synthesis. We'll focus on minimizing latency.

## Step 1: Install and Import Required Libraries

In [1]:
!pip install faster-whisper
!pip install huggingface_hub
!pip install edge-tts
!pip install dspy
!pip install torch
!pip install sounddevice

^C
  Using cached edge_tts-6.1.12-py3-none-any.whl.metadata (4.0 kB)
  Using cached aiohttp-3.10.5-cp311-cp311-win_amd64.whl.metadata (7.8 kB)
  Using cached aiohappyeyeballs-2.4.0-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached yarl-1.9.4-cp311-cp311-win_amd64.whl.metadata (32 kB)
Using cached edge_tts-6.1.12-py3-none-any.whl (29 kB)
Using cached aiohttp-3.10.5-cp311-cp311-win_amd64.whl (379 kB)
Using cached aiohappyeyeballs-2.4.0-py3-none-any.whl (12 kB)
Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
Using cached yarl-1.9.4-cp311-cp311-win_amd64.whl (76 kB)
  Using cached faster_whisper-1.0.3-py3-none-any.whl.metadata (15 kB)
  Using cached av-12.3.0-cp311-cp311-win_amd64.whl.metadata (4.7 kB)
  Using cached ctranslate2-4.3.1-cp311-cp311-win_amd64.whl.metadata (10 kB)
  Using cached tokenizers-0.20.0-cp311-none-win_amd64.whl.metadata (6.9 kB)
  Using cached onnxruntime-1.19.0-cp311-cp311-win_amd64.whl.metadat

## Step 2: Import Libraries and Initialize Models

In [2]:
import sounddevice as sd
import numpy as np
from faster_whisper import WhisperModel
from huggingface_hub import InferenceApi
from dspy import DynamicPrompt
import edge_tts
import asyncio

# Initialize Whisper model
model = WhisperModel('small', device='cuda', compute_type='int8_float16')

# Initialize the Inference API with Mistral model
inference = InferenceApi(repo_id='mistralai/Mistral-7B-Instruct', token='YOUR_HF_API_KEY')

# Function to Capture Audio
def capture_audio(duration=5, fs=16000):
    print('Recording...')
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='float32')
    sd.wait()  # Wait until recording is finished
    return audio.flatten()

# Function to Apply Voice Activity Detection (VAD)
def apply_vad(audio, threshold=0.5):
    vad_audio = []
    for chunk in np.array_split(audio, len(audio) // int(0.02 * 16000)):
        if np.mean(np.abs(chunk)) > threshold:
            vad_audio.extend(chunk)
    return np.array(vad_audio)


OSError: [WinError 126] The specified module could not be found. Error loading "c:\Users\Asus\OneDrive\Desktop\Hackathon_project\Hack4change\TranscendAI\.conda\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

## Step 3: Generate LLM Response

In [ ]:
# Define the LLM Response Function
def generate_response(prompt):
    response = inference(inputs=prompt, parameters={'max_length': 20})
    return response.get('generated_text', '')

# Example usage with Dynamic Prompting
def generate_dynamic_prompt(query):
    dp = DynamicPrompt()
    dp.add_prompt(f'Q: {query} A:', weight=1.0)
    dp.add_context('You are a helpful assistant.', weight=0.8)
    return dp.render()


## Step 4: Text-to-Speech Conversion with Tunable Parameters

In [ ]:
# Function to Convert Text to Speech
async def text_to_speech(text, voice='en-US-JennyNeural', rate='+0%', pitch='+0%'):
    communicate = edge_tts.Communicate(text, voice=voice, rate=rate, pitch=pitch)
    await communicate.save('output_audio.mp3')

# Full pipeline execution example
audio = capture_audio()
vad_audio = apply_vad(audio)
text_result = model.transcribe(vad_audio, vad_threshold=0.5)
transcribed_text = text_result['text']
print('Transcribed Text:', transcribed_text)

dynamic_prompt = generate_dynamic_prompt(transcribed_text)
response = generate_response(dynamic_prompt)
print('LLM Response:', response)

# Convert the response to speech
asyncio.run(text_to_speech(response, voice='en-US-GuyNeural', rate='+5%', pitch='-2%'))

## Summary
- **Voice-to-Text:** Using Whisper with VAD for efficient transcription.
- **Text Processing:** Mistral model via Hugging Face Inference API for low latency and fast response.
- **Prompt Engineering:** Dynamic prompting with `dspy` for enhanced responses.
- **Text-to-Speech:** Tunable TTS for custom pitch, rate, and voice type.